# Import libraries

In [4]:
import pandas as pd
import numpy as np
import xlwings as xw
import matplotlib.pyplot as plt

# Define global variables

In [5]:
xlDataName = '\DissData [02]'
dataFolder = r"C:\Users\rfg\OneDrive\Desktop\Dissertation ES30029\data"
companyNamesRange = "A1:A2009"
cellRng = "A1:BYH288"
xlBook = xw.books(xlDataName[1:])
sheetWhitelist = ['all_companies']
fields = []

# Read data from Excel

In [31]:
fields = []
for sheet in xlBook.sheets:
    if sheet.name not in sheetWhitelist:
        exec('{} = pd.DataFrame(xw.books(xlDataName[1:]).sheets("{}").range(cellRng).value)'.format(sheet.name, sheet.name))
        exec('fields.append({})'.format(sheet.name))
        


# Clean up the dataframes to have correct cols and rows
for field in fields:
    companyNames = [str(i) for i in xlBook.sheets("all_companies").range(companyNamesRange).value]
    companyNames.insert(0, field.iloc[0][0])
    field.columns = companyNames
    field.index = field[field.columns[0]]
    del field[field.columns[0]]
    field.drop(field.index[0], inplace=True)
    field.replace("NA", np.nan, inplace=True)
    field = field.apply(pd.to_numeric,errors='coerce')
    


# Data Cleaning

### Remove series which don't have data for a field

In [32]:
errorCols = []
for field in fields:
    for col in field:
        try:
            if field[col].iloc[0][:4] == '$$ER': 
                errorCols.append(col)
        except:
            pass

errorCols = list(dict.fromkeys(errorCols))

for field in fields:
    for col in errorCols:
        del field[col]

print(np.stack(fields, axis=-1).shape)

# Data Calculation

### Returns

In [34]:
r = p/p.shift(1)-1

In [35]:
ret_l3m = (1+r).rolling(window=3).apply(np.prod, raw=True)-1
ret_l6m = (1+r).rolling(window=6).apply(np.prod, raw=True)-1
ret_l9m = (1+r).rolling(window=9).apply(np.prod, raw=True)-1
ret_l12m = (1+r).rolling(window=12).apply(np.prod, raw=True)-1
ret_l18m = (1+r).rolling(window=18).apply(np.prod, raw=True)-1
ret_l24m = (1+r).rolling(window=24).apply(np.prod, raw=True)-1
ret_l36m = (1+r).rolling(window=36).apply(np.prod, raw=True)-1

rollingReturns = [ret_l3m, ret_l6m, ret_l9m, ret_l12m, ret_l18m, ret_l24m, ret_l36m]
for i in rollingReturns:
    print(i.values.shape)

for i in rollingReturns:
    fields.append(i)

(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)


### Moving averages

In [36]:
# Calculate moving averages
p_ma3 = p.rolling(window=3).mean()
p_ma6 = p.rolling(window=6).mean()
p_ma9 = p.rolling(window=9).mean()
p_ma12 = p.rolling(window=12).mean()
p_ma18 = p.rolling(window=18).mean()
p_ma24 = p.rolling(window=24).mean()
p_ma36 = p.rolling(window=36).mean()

# Technical indicators (crosses of MAVs for example)

movingAverages = [p_ma3, p_ma6, p_ma9, p_ma12, p_ma18, p_ma24, p_ma36]
for i in movingAverages:
    print(i.values.shape)

for i in movingAverages:
    fields.append(i)

(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)


### Volatilities

In [37]:
std_3m = r.rolling(window=3).std() * (12 ** 0.5)
std_6m = r.rolling(window=6).std() * (12 ** 0.5)
std_9m = r.rolling(window=9).std() * (12 ** 0.5)
std_12m = r.rolling(window=12).std() * (12 ** 0.5)
std_18m = r.rolling(window=18).std() * (12 ** 0.5)
std_24m = r.rolling(window=24).std() * (12 ** 0.5)
std_36m = r.rolling(window=36).std() * (12 ** 0.5)

volatilities = [std_3m, std_6m, std_9m, std_12m, std_18m, std_24m, std_36m]
for i in volatilities:
    print(i.values.shape)

for i in volatilities:
    fields.append(i)

(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)
(287, 1562)


# Export Data

### Numpy file

In [38]:
# Also save index and columns so that you can load up data with these
np.save(file=dataFolder+r'\npData.npy', arr=np.stack(fields, axis=-1))
np.save(file=dataFolder+'\index.npy', arr=p.index)
np.save(file=dataFolder+'\columns.npy', arr=p.columns)

In [39]:
print(np.stack(fields, axis=-1).shape)

(287, 1562, 33)
